# Onclusive Machine Learning Challenge
## Build an ML system to verify the veracity of claims in dataset PUBHEALTH.
### Outlining the steps and demonstrate the performance of the final model.  

In [2]:
import numpy as np
import pandas as pd
from datasets import load_dataset, load_metric
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding

In [3]:
dataset = load_dataset("health_fact")

Using custom data configuration default
Reusing dataset health_fact (C:\Users\david\.cache\huggingface\datasets\health_fact\default\1.1.0\99503637e4255bd805f84d57031c18fe4dd88298f00299d56c94fc59ed68ec19)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
dataset = dataset.filter(lambda example: example['label'] != -1)

Loading cached processed dataset at C:\Users\david\.cache\huggingface\datasets\health_fact\default\1.1.0\99503637e4255bd805f84d57031c18fe4dd88298f00299d56c94fc59ed68ec19\cache-41180d96da5bd5f3.arrow
Loading cached processed dataset at C:\Users\david\.cache\huggingface\datasets\health_fact\default\1.1.0\99503637e4255bd805f84d57031c18fe4dd88298f00299d56c94fc59ed68ec19\cache-e37781bb9e63676d.arrow
Loading cached processed dataset at C:\Users\david\.cache\huggingface\datasets\health_fact\default\1.1.0\99503637e4255bd805f84d57031c18fe4dd88298f00299d56c94fc59ed68ec19\cache-bd791c5953b7d1ab.arrow


In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [6]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["main_text"], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Parameter 'function'=<function tokenize_function at 0x0000012FBF15A790> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [8]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=40).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=40).select(range(100))

In [9]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=4)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

In [10]:
training_args = TrainingArguments(output_dir="model/distilbert-base-uncased")

In [33]:
metric = load_metric("f1")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average=average)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    # train_dataset=small_train_dataset,
    # eval_dataset=small_eval_dataset,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [13]:
old_collator = trainer.data_collator
trainer.data_collator = lambda data: dict(old_collator(data))

In [14]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: subjects, date_published, sources, claim, explanation, claim_id, main_text, fact_checkers.
***** Running training *****
  Num examples = 9804
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3678


  0%|          | 0/3678 [00:00<?, ?it/s]

Saving model checkpoint to distilbert-base-uncased\checkpoint-500
Configuration saved in distilbert-base-uncased\checkpoint-500\config.json


{'loss': 0.8413, 'learning_rate': 4.320282762370854e-05, 'epoch': 0.41}


Model weights saved in distilbert-base-uncased\checkpoint-500\pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased\checkpoint-500\tokenizer_config.json
Special tokens file saved in distilbert-base-uncased\checkpoint-500\special_tokens_map.json
Saving model checkpoint to distilbert-base-uncased\checkpoint-1000
Configuration saved in distilbert-base-uncased\checkpoint-1000\config.json


...

Model weights saved in distilbert-base-uncased\checkpoint-3000\pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased\checkpoint-3000\tokenizer_config.json
Special tokens file saved in distilbert-base-uncased\checkpoint-3000\special_tokens_map.json
Saving model checkpoint to distilbert-base-uncased\checkpoint-3500
Configuration saved in distilbert-base-uncased\checkpoint-3500\config.json


{'loss': 0.4262, 'learning_rate': 2.419793365959761e-06, 'epoch': 2.85}


Model weights saved in distilbert-base-uncased\checkpoint-3500\pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased\checkpoint-3500\tokenizer_config.json
Special tokens file saved in distilbert-base-uncased\checkpoint-3500\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 707.0239, 'train_samples_per_second': 41.6, 'train_steps_per_second': 5.202, 'train_loss': 0.618635307258079, 'epoch': 3.0}


TrainOutput(global_step=3678, training_loss=0.618635307258079, metrics={'train_runtime': 707.0239, 'train_samples_per_second': 41.6, 'train_steps_per_second': 5.202, 'train_loss': 0.618635307258079, 'epoch': 3.0})

In [15]:
trainer.save_model()

Saving model checkpoint to distilbert-base-uncased
Configuration saved in distilbert-base-uncased\config.json
Model weights saved in distilbert-base-uncased\pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased\tokenizer_config.json
Special tokens file saved in distilbert-base-uncased\special_tokens_map.json


In [41]:
average=None
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: subjects, date_published, sources, claim, explanation, claim_id, main_text, fact_checkers.
***** Running Evaluation *****
  Num examples = 1214
  Batch size = 8


  0%|          | 0/152 [00:00<?, ?it/s]

Trainer is attempting to log a value of "[0.66666667 0.35474006 0.85498489 0.34782609]" of type <class 'numpy.ndarray'> for key "eval_f1" as a metric. MLflow's log_metric() only accepts float and int types so we dropped this attribute.
Trainer is attempting to log a value of "[0.66666667 0.35474006 0.85498489 0.34782609]" of type <class 'numpy.ndarray'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.836951494216919,
 'eval_f1': array([0.66666667, 0.35474006, 0.85498489, 0.34782609]),
 'eval_runtime': 9.3677,
 'eval_samples_per_second': 129.595,
 'eval_steps_per_second': 16.226,
 'epoch': 3.0}

In [42]:
average='macro'
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: subjects, date_published, sources, claim, explanation, claim_id, main_text, fact_checkers.
***** Running Evaluation *****
  Num examples = 1214
  Batch size = 8


  0%|          | 0/152 [00:00<?, ?it/s]

{'eval_loss': 0.836951494216919,
 'eval_f1': 0.5560544272612717,
 'eval_runtime': 9.3073,
 'eval_samples_per_second': 130.435,
 'eval_steps_per_second': 16.331,
 'epoch': 3.0}

In [43]:
average='micro'
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: subjects, date_published, sources, claim, explanation, claim_id, main_text, fact_checkers.
***** Running Evaluation *****
  Num examples = 1214
  Batch size = 8


  0%|          | 0/152 [00:00<?, ?it/s]

{'eval_loss': 0.836951494216919,
 'eval_f1': 0.7182866556836902,
 'eval_runtime': 9.1615,
 'eval_samples_per_second': 132.51,
 'eval_steps_per_second': 16.591,
 'epoch': 3.0}

In [44]:
average='weighted'
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: subjects, date_published, sources, claim, explanation, claim_id, main_text, fact_checkers.
***** Running Evaluation *****
  Num examples = 1214
  Batch size = 8


  0%|          | 0/152 [00:00<?, ?it/s]

{'eval_loss': 0.836951494216919,
 'eval_f1': 0.71133201929041,
 'eval_runtime': 9.3761,
 'eval_samples_per_second': 129.478,
 'eval_steps_per_second': 16.211,
 'epoch': 3.0}

In [46]:
average='macro'
trainer.predict(test_dataset=tokenized_datasets['test'])

The following columns in the test set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: subjects, date_published, sources, claim, explanation, claim_id, main_text, fact_checkers.
***** Running Prediction *****
  Num examples = 1233
  Batch size = 8


PredictionOutput(predictions=array([[ 1.0322939 , -0.29301193, -1.4410217 ,  0.47513568],
       [-0.9447125 , -2.1634445 ,  4.8928523 , -2.9590476 ],
       [ 2.8517153 ,  1.210928  , -1.4673307 , -3.7730813 ],
       ...,
       [ 2.4935594 ,  0.96268743, -0.77854323, -3.8964727 ],
       [-0.55611557, -1.3674632 ,  4.1762085 , -3.5121956 ],
       [-0.7599856 , -2.1547089 ,  4.585301  , -2.8459973 ]],
      dtype=float32), label_ids=array([0, 2, 2, ..., 0, 2, 2], dtype=int64), metrics={'test_loss': 0.929521381855011, 'test_f1': 0.5523390297624453, 'test_runtime': 9.3581, 'test_samples_per_second': 131.758, 'test_steps_per_second': 16.563})